<a href="https://colab.research.google.com/github/demelere/Weld-Seam-Finding/blob/main/FastSAM_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/demelere/FastSAM.git

Cloning into 'FastSAM'...
remote: Enumerating objects: 1329, done.
remote: Counting objects: 100% (401/401), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 1329 (delta 346), reused 308 (delta 308), pack-reused 928 (from 1)
Receiving objects: 100% (1329/1329), 72.56 MiB | 15.46 MiB/s, done.
Resolving deltas: 100% (542/542), done.


In [2]:
!nvidia-smi

Mon Apr 14 21:22:45 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   45C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
!wget https://huggingface.co/spaces/An-619/FastSAM/resolve/main/weights/FastSAM.pt

--2025-04-14 21:22:45--  https://huggingface.co/spaces/An-619/FastSAM/resolve/main/weights/FastSAM.pt
Resolving huggingface.co (huggingface.co)... 108.138.246.79, 108.138.246.85, 108.138.246.71, ...
Connecting to huggingface.co (huggingface.co)|108.138.246.79|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/71/c3/71c3ec5a83f3fc374665cec9240f614262e29088fca1a18732571e13c7084f1b/c0be4e7ddbe4c15333d15a859c676d053c486d0a746a3be6a7a9790d52a9b6d7?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27FastSAM.pt%3B+filename%3D%22FastSAM.pt%22%3B&Expires=1744669365&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0NDY2OTM2NX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy83MS9jMy83MWMzZWM1YTgzZjNmYzM3NDY2NWNlYzkyNDBmNjE0MjYyZTI5MDg4ZmNhMWExODczMjU3MWUxM2M3MDg0ZjFiL2MwYmU0ZTdkZGJlNGMxNTMzM2QxNWE4NTljNjc2ZDA1M2M0ODZkMGE3NDZhM2JlNmE3YTk3OTBkNTJhOWI2ZDc%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3

In [4]:
!pip install -r FastSAM/requirements.txt
!pip install git+https://github.com/openai/CLIP.git

INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/

In [5]:
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np
from PIL import Image
from google.colab import files
# !wget -P images https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/dog.jpg
uploaded = files.upload()

Saving tee-joint.jpg to tee-joint.jpg


In [14]:
print(os.listdir())

['.config', 'FastSAM.pt', 'tee-joint.jpg', 'FastSAM', 'sample_data']


In [13]:
# import os

# !os.makedirs('images', exist_ok=True)
# !os.makedirs('output', exist_ok=True)

# # Save the uploaded file temporarily
# for filename in uploaded.keys():
#     # Save the uploaded file to the images directory
#     with open(os.path.join('images', filename), 'wb') as f:
#         f.write(uploaded[filename])

#     # Now you can use the saved file path for inference
#     input_path = os.path.join('images', filename)

/bin/bash: -c: line 1: syntax error near unexpected token `'images','
/bin/bash: -c: line 1: `os.makedirs('images', exist_ok=True)'
/bin/bash: -c: line 1: syntax error near unexpected token `'output','
/bin/bash: -c: line 1: `os.makedirs('output', exist_ok=True)'


FileNotFoundError: [Errno 2] No such file or directory: 'images/tee-joint.jpg'

In [ ]:
os.makedirs('./output', exist_ok=True)

In [ ]:
image_path = './tee-joint.jpg'
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [ ]:
original_h = image.shape[0]
original_w = image.shape[1]
print(original_w, original_h)
plt.figure(figsize=(10, 10))
plt.imshow(image)

"--imgsz 1024" means the input image size is 1024. Our model was trained on a size of 1024. You can change it to any size you want to input. Using other sizes would produce different segmentation results.

In [ ]:
!python FastSAM/Inference.py --model_path FastSAM.pt --img_path ./tee-joint.jpg --imgsz 1024

In [ ]:
output_path = './output/tee-joint.jpg'
result_image = cv2.imread(output_path)
result_image = cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB)
original_h = result_image.shape[0]
original_w = result_image.shape[1]
print(original_w, original_h)
plt.figure(figsize=(10, 10))
plt.axis("off")
plt.imshow(result_image)